In [2]:
import pandas as pd
import numpy as np
import glob
import nibabel as nib
from scipy.ndimage import label
from numpy.linalg import inv

## msid + mseid data

In [44]:
#Construct data table
final_cols = ["msid", #done
             "tot_years_gt1_lesion_present", #done
             "inf_years_gt1_lesion_present", #done
             "jux_years_gt1_lesion_present", #done
             "per_years_gt1_lesion_present", #done
             "sub_years_gt1_lesion_present", #done
             "tot_lesion_presentation", #done
             "inf_lesion_presentation", #done
             "jux_lesion_presentation", #done
             "per_lesion_presentation", #done
             "sub_lesion_presentation", #done
             "comparison", #done
             "corresponding_mseids", #done
             "tot_lesion_change_percent", #done
             "tot_lesion_change_absolute", #done
             "tot_lesion_vol_change_percent", #done
             "tot_lesion_vol_change_absolute", #done
             "inf_lesion_change_percent", #done
             "inf_lesion_change_absolute", #done
             "inf_lesion_vol_change_percent", #done
             "inf_lesion_vol_change_absolute", #done
             "jux_lesion_change_percent", #done
             "jux_lesion_change_absolute", #done
             "jux_lesion_vol_change_percent", #done
             "jux_lesion_vol_change_absolute", #done
             "per_lesion_change_percent", #done
             "per_lesion_change_absolute", #done
             "per_lesion_vol_change_percent", #done
             "per_lesion_vol_change_absolute", #done
             "sub_lesion_change_percent", #done
             "sub_lesion_change_absolute", #done
             "sub_lesion_vol_change_percent", #done
             "sub_lesion_vol_change_absolute", #done
             "tot_number_increasing_0",
             "tot_number_increasing_10",
             "tot_number_increasing_20",
             "tot_number_increasing_50",
             "tot_number_increasing_100",
             "inf_number_increasing_0",
             "inf_number_increasing_10",
             "inf_number_increasing_20",
             "inf_number_increasing_50",
             "inf_number_increasing_100",
             "jux_number_increasing_0",
             "jux_number_increasing_10",
             "jux_number_increasing_20",
             "jux_number_increasing_50",
             "jux_number_increasing_100",
             "per_number_increasing_0",
             "per_number_increasing_10",
             "per_number_increasing_20",
             "per_number_increasing_50",
             "per_number_increasing_100",
             "sub_number_increasing_0",
             "sub_number_increasing_10",
             "sub_number_increasing_20",
             "sub_number_increasing_50",
             "sub_number_increasing_100",
             "tot_number_decreasing_0",
             "tot_number_decreasing_10",
             "tot_number_decreasing_20",
             "tot_number_decreasing_50",
             "tot_number_decreasing_100",
             "inf_number_decreasing_0",
             "inf_number_decreasing_10",
             "inf_number_decreasing_20",
             "inf_number_decreasing_50",
             "inf_number_decreasing_100",
             "jux_number_decreasing_0",
             "jux_number_decreasing_10",
             "jux_number_decreasing_20",
             "jux_number_decreasing_50",
             "jux_number_decreasing_100",
             "per_number_decreasing_0",
             "per_number_decreasing_10",
             "per_number_decreasing_20",
             "per_number_decreasing_50",
             "per_number_decreasing_100",
             "sub_number_decreasing_0",
             "sub_number_decreasing_10",
             "sub_number_decreasing_20",
             "sub_number_decreasing_50",
             "sub_number_decreasing_100",
             "tot_new_lesions",
             "tot_lost_lesions",
             "inf_new_lesions",
             "inf_lost_lesions",
             "jux_new_lesions",
             "jux_lost_lesions",
             "per_new_lesions",
             "per_lost_lesions",
             "sub_new_lesions",
             "sub_lost_lesions",
             "lesions",
             "coordinates",
             "overlap_1",
             "overlap_2",
             "volume_change_absolute",
             "volume_change_percent",
             "change_in_lesion_type"]
analysis = {}
for item in final_cols:
    analysis[item] = []
les_data = pd.DataFrame.from_csv('/data/henry1/mahamber/lesion_info_baseline.csv').drop(['total number of lesions', 'subcortical lesions',
       'juxtacortical lesions', 'periventricular lesions','infratentorial lesions', 'center coordinates',
       'volume', 'distance from midbrain', 'distance from ventricles','distance from gray matter'], axis=1)
prev_data = pd.DataFrame.from_csv('clinical_info.csv')

,mseID,lesion,type
213,mse1,1,juxtacortical
214,mse1,2,infratentorial
215,mse1,3,juxtacortical
216,mse1,4,subcortical
217,mse1,5,juxtacortical
218,mse1,6,periventricular
219,mse1,7,periventricular
220,mse1,8,subcortical
221,mse1,9,periventricular
222,mse1,10,juxtacortical


In [29]:
#created matrix of tps
tp_matrix = {}
count = 1
for x in range(1,10):
    for y in range(x+1,11):
        tp_matrix["%s,%s" %(x,y)]=count
        count += 1
tp_matrix

#create sorted DataFrame in order of msid and tp
#create list of msids for looping purposes
df = pd.DataFrame(prev_data).sort_values(by=['msid','tp']).reset_index()
cols = pd.Index(df.columns)
msids = df.msid.values
types = ['tot','inf','jux','per','sub']

#percent change function
def percentchange(num2,num1):
    if num1 == 0 or num2 == 0:
        value = 0
    else:
        value = ((float(num2)-float(num1))/(float(num1))) * 100
    return round(value,2)

#lesion presentation pattern determination function
def pattern(lesionlist):
    truth = []
    for x in lesionlist:
        if x >= 1:
            truth.append(True)
        else:
            truth.append(False)
    if truth.count(True) == len(lesionlist):
        return "throughout"
    elif truth.count(False) == len(lesionlist):
        return "never"
    else:
        if truth[0] == False:
            return "late"
        elif truth[-1] == False:
            return "early"
        else: 
            return "intermittent"

#gather lesion coordinates for labels
def get_lesion_info(mseid):
    les_src = glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % mseid)
    if len(les_src) != 0:
        les_file = les_src[0]
        les_label_num = 1
    else:
        les_src = glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/*lesions.nii.gz" % mseid)
        if len(les_src) != 0:
            les_file = les_src[0]
            les_label_num = 1
        else:
            return
    les_img = nib.load(les_file)
    les_data = les_img.get_data()
    les_affine = les_img.get_affine()
    labels, n_labels = label(les_data==les_label_num)
    return labels, n_labels
    
def get_label_coord(labels,num_labels):
    all_label_coords = []
    for count in range(1,num_labels+1):
        x,y,z = np.nonzero(labels==count)
        x_coords=[];y_coords=[];z_coords=[]
        for count in range(len(x)):
            x_coords.append(x[count])
            y_coords.append(y[count])
            z_coords.append(z[count])
        all_label_coords.append([x_coords,y_coords,z_coords])
    return all_label_coords

#meat of the program
for ids in msids:
    #collect values per tp per msid 
    #specifically collect mseids, total lesion count, total lesion volume, total lesion count (type), total lesion volume (type)
    msid_vals = {}
    current_msid = df['msid']==ids
    tp_list = df[current_msid]['tp'].values 
    mseid_list = df[current_msid]['mse'].values
    for les in types:
        msid_vals['%s_tot_list' % les] = df[current_msid]['%s_total_lesions' % les].values
        msid_vals['%s_vol_list' % les] = df[current_msid]['%s_total_volume_of_lesions' % les].values    
    
    #calculate number of year by year comparisons needed for each msid
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(yrx+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])
            
    #calculate totals, changes for each lesion type and entire brain
    for les in types:
        print les, msid_vals['%s_tot_list' % les], msid_vals['%s_vol_list' % les],
        msid_vals['%s_change_list' % les] = []
        for comp in range(len(tp_ix_comp)):
            les_tot_change_abs = msid_vals['%s_tot_list' % les][tp_ix_comp[comp][1]] - msid_vals['%s_tot_list' % les][tp_ix_comp[comp][0]]
            les_tot_change_per = percentchange(msid_vals['%s_tot_list' % les][tp_ix_comp[comp][1]],msid_vals['%s_tot_list' % les][tp_ix_comp[comp][0]])
            les_vol_change_abs = msid_vals['%s_vol_list' % les][tp_ix_comp[comp][1]] - msid_vals['%s_vol_list' % les][tp_ix_comp[comp][0]]
            les_tot_change_per = percentchange(msid_vals['%s_vol_list' % les][tp_ix_comp[comp][1]],msid_vals['%s_vol_list' % les][tp_ix_comp[comp][0]])
            msid_vals['%s_change_list' % les].append([les_tot_change_abs,les_tot_change_per,les_vol_change_abs,les_vol_change_per])
    print
    
    #append calculations to dictionary for dataframe creation
    for count in range(len(msid_vals['tot_change_list'])):
        for les in types:
            #append absolute and percent change statistics
            analysis["%s_lesion_change_percent" % les].append(msid_vals['%s_change_list' % les][count][1])
            analysis["%s_lesion_change_absolute" % les].append(msid_vals['%s_change_list' % les][count][0])
            analysis["%s_lesion_vol_change_percent" % les].append(msid_vals['%s_change_list' % les][count][3])
            analysis["%s_lesion_vol_change_absolute" % les].append(msid_vals['%s_change_list' % les][count][2])
            
            #append counts for years of comparison
            total1 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][0]]
            total2 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][1]]
            analysis["%s_lesion_counts" % les].append([total1,total2])
            
            #find out and append how many years greater than 1 lesion present for each lesion type + total lesions
            xcount = 0
            for les_count in range(len(msid_vals['%s_tot_list' % les])):
                if msid_vals['%s_tot_list' % les][les_count] == 0:
                    xcount += 1
            analysis["%s_years_gt1_lesion_present" % les].append(count)
            
            #determine presentation pattern (throughout, never, came late, left early, intermittent)
            analysis["%s_lesion_presentation" % les].append(pattern(msid_vals['%s_tot_list' % les]))
        
        #append mkid and years of comparison
        analysis["msid"].append(testids[x])
        analysis["comparison"].append(tp_yr_comp[count])
        analysis["corresponding_mseids"].append([mseid_list[tp_ix_comp[count][0]], mseid_list[tp_ix_comp[count][1]]])
        print analysis["corresponding_mseids"]
    
    #print values for debugging purposes
change_analysis = pd.DataFrame(analysis,columns=final_cols)
change_analysis

tot [11] [ 752.] inf [0] [ 0.] jux [0] [ 0.] per [1] [ 61.] sub [10] [ 691.]
tot [2] [ 408.] inf [1] [ 83.] jux [0] [ 0.] per [0] [ 0.] sub [1] [ 325.]
tot [14] [ 6618.] inf [1] [ 22.] jux [3] [ 534.] per [4] [ 3661.] sub [6] [ 2401.]
tot [14] [ 27988.] inf [1] [ 54.] jux [4] [ 189.] per [5] [ 27487.] sub [4] [ 258.]
tot [37] [ 10159.] inf [0] [ 0.] jux [11] [ 957.] per [10] [ 7461.] sub [16] [ 1741.]
tot [19] [ 11583.] inf [1] [ 356.] jux [2] [ 242.] per [9] [ 5003.] sub [7] [ 5982.]
tot [4] [ 760.] inf [0] [ 0.] jux [1] [ 38.] per [2] [ 500.] sub [1] [ 222.]
tot [19] [ 7223.] inf [0] [ 0.] jux [4] [ 595.] per [11] [ 6156.] sub [4] [ 472.]
tot [25] [ 4774.] inf [0] [ 0.] jux [6] [ 567.] per [3] [ 1318.] sub [16] [ 2889.]
tot [15] [ 11904.] inf [0] [ 0.] jux [2] [ 1270.] per [8] [ 9881.] sub [5] [ 753.]
tot [38] [ 6272.] inf [1] [ 279.] jux [9] [ 1492.] per [12] [ 1862.] sub [16] [ 2639.]
tot [4] [ 732.] inf [0] [ 0.] jux [0] [ 0.] per [2] [ 539.] sub [2] [ 193.]
tot [11] [ 1126.] inf 

,msid,tot_years_gt1_lesion_present,inf_years_gt1_lesion_present,jux_years_gt1_lesion_present,per_years_gt1_lesion_present,sub_years_gt1_lesion_present,tot_lesion_presentation,inf_lesion_presentation,jux_lesion_presentation,per_lesion_presentation,...,per_lost_lesions,sub_new_lesions,sub_lost_lesions,lesions,coordinates,overlap_1,overlap_2,volume_change_absolute,volume_change_percent,change_in_lesion_type


## Dummy Code

In [25]:
import random

#created test data and dataframe, sorted by mkid and tp
test = {"mkid" : ["mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4","mk1","mk2","mk3","mk4"],
        "mkeid" : ["mke009","mke002","mke003","mke004","mke005","mke006","mke007","mke008",
                   "mke001","mke010","mke011","mke012"],
        "tp" : [1,1,1,1,2,2,2,3,3,3,5,4],
        "tot_total_lesions" : [5,10,6,14,1,11,24,13,25,27,10,4],
        "sub_total_lesions" : [1,5,0,5,1,0,11,3,12,9,3,2],
        "inf_total_lesions" : [1,2,2,3,0,1,4,3,25,4,3,0],
        "per_total_lesions" : [2,2,4,4,0,5,10,3,4,10,3,1],
        "jux_total_lesions" : [1,1,0,4,0,4,0,4,5,4,1,1],
       }

coords = range(80,130)
size = [50,45]
lesions = []
for x in size:
    lesion_vol_x = random.sample(coords,x)
    lesion_vol_y = random.sample(coords,x)
    lesion_vol_z = random.sample(coords,x)
    lesions.append([lesion_vol_x,lesion_vol_y,lesion_vol_z])
a = min(len(lesions[0][0]),len(lesions[1][0]))
b = max(len(lesions[0][0]),len(lesions[1][0]))
print a, b
for x in range(a):
    print lesions[0][0][x], lesions[0][1][x], lesions[0][2][x]

45 50
80 89 94
94 86 129
90 92 86
91 95 82
101 113 85
123 120 125
93 125 124
89 100 101
114 117 96
116 103 109
84 83 121
129 98 93
99 97 103
81 107 81
125 80 117
118 81 126
113 108 100
121 126 87
86 90 92
106 123 84
127 112 114
117 127 128
120 124 120
97 111 98
82 122 119
111 93 116
85 119 88
108 99 83
119 110 104
107 106 112
92 88 105
124 102 102
83 96 123
87 118 99
96 129 122
98 121 90
122 128 95
126 101 111
88 104 110
105 114 127
110 116 113
102 82 108
112 105 118
95 109 80
100 91 91


In [26]:
testcolumns = test.keys()
testdf = pd.DataFrame(test, columns=[testcolumns]).sort_values(by=['mkid','tp'])
testids = testdf.mkid.values
types = ['tot','inf','jux','per','sub']


#analysis dataframe
analysiscol=["mkid",
             "tot_years_gt1_lesion_present",
             "inf_years_gt1_lesion_present",
             "jux_years_gt1_lesion_present",
             "per_years_gt1_lesion_present",
             "sub_years_gt1_lesion_present",
             "tot_lesion_presentation",
             "inf_lesion_presentation",
             "jux_lesion_presentation",
             "per_lesion_presentation",
             "sub_lesion_presentation",
             "comparison",
             "tot_lesion_counts",
             "tot_lesion_change_percent",
             "tot_lesion_change_absolute",
             "inf_lesion_counts",
             "inf_lesion_change_percent",
             "inf_lesion_change_absolute",
             "jux_lesion_counts",
             "jux_lesion_change_percent",
             "jux_lesion_change_absolute",
             "per_lesion_counts",
             "per_lesion_change_percent",
             "per_lesion_change_absolute",
             "sub_lesion_counts",
             "sub_lesion_change_percent",
             "sub_lesion_change_absolute"]
analysis = {}
for item in analysiscol:
    analysis[item] = []

#lesion presentation pattern determination function
def pattern(lesionlist):
    truth = []
    for x in lesionlist:
        if x >= 1:
            truth.append(True)
        else:
            truth.append(False)
    if truth.count(True) == len(lesionlist):
        return "throughout"
    elif truth.count(False) == len(lesionlist):
        return "never"
    else:
        if truth[0] == False:
            return "late"
        elif truth[-1] == False:
            return "early"
        else: 
            return "intermittent"
        
#perform calculations
print "mkid", "comparison", "abs_lesion_change", "per_lesion_change"
for x in range(0,len(testids),3):
    #collect values mkid,mkeid,tp
    msid_vals = {}
    val = testdf['mkid']==testids[x] #shortcut to gathering variables by mkid
    tp_list = testdf[val]['tp'].values 
    mkeid_list = testdf[val]['mkeid'].values

    #collect tp yrs and ixs for change analysis
    tp_yr_comp = []
    tp_ix_comp = []
    for yrx in range(len(tp_list)):
        for yry in range(yrx+1,len(tp_list)):
            tp_yr_comp.append("%s,%s" % (tp_list[yrx],tp_list[yry]))
            tp_ix_comp.append([yrx,yry])

    #calculate totals, changes for each lesion type and entire brain
    for les in range(len(types)):
        msid_vals['%s_tot_list' % types[les]] = testdf[val]['%s_total_lesions' % types[les]].values
        msid_vals['%s_change_list' % types[les]] = []
        for comp in range(len(tp_ix_comp)):
            les_change_abs = msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]] - msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]]
            les_change_per = percentchange(msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][1]],msid_vals['%s_tot_list' % types[les]][tp_ix_comp[comp][0]])
            msid_vals['%s_change_list' % types[les]].append([les_change_abs,les_change_per])

    #append calculations to dictionary for dataframe creation
    for count in range(len(msid_vals['tot_change_list'])):
        for les in types:
            #append absolute and percent change statistics
            analysis["%s_lesion_change_percent" % les].append(msid_vals['%s_change_list' % les][count][1])
            analysis["%s_lesion_change_absolute" % les].append(msid_vals['%s_change_list' % les][count][0])
            
            #append counts for years of comparison
            total1 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][0]]
            total2 = msid_vals['%s_tot_list' % les][tp_ix_comp[count][1]]
            analysis["%s_lesion_counts" % les].append([total1,total2])
            
            #find out and append how many years greater than 1 lesion present for each lesion type + total lesions
            xcount = 0
            for les_count in range(len(msid_vals['%s_tot_list' % les])):
                if msid_vals['%s_tot_list' % les][les_count] == 0:
                    xcount += 1
            analysis["%s_years_gt1_lesion_present" % les].append(xcount)
            
            #determine presentation pattern (throughout, never, came late, left early, intermittent)
            analysis["%s_lesion_presentation" % les].append(pattern(msid_vals['%s_tot_list' % les]))
        
        #append mkid and years of comparison
        analysis["mkid"].append(testids[x])
        analysis["comparison"].append(tp_yr_comp[count])

lulz = pd.DataFrame(analysis,columns=analysiscol)
lulz

mkid comparison abs_lesion_change per_lesion_change


,mkid,tot_years_gt1_lesion_present,inf_years_gt1_lesion_present,jux_years_gt1_lesion_present,per_years_gt1_lesion_present,sub_years_gt1_lesion_present,tot_lesion_presentation,inf_lesion_presentation,jux_lesion_presentation,per_lesion_presentation,...,inf_lesion_change_absolute,jux_lesion_counts,jux_lesion_change_percent,jux_lesion_change_absolute,per_lesion_counts,per_lesion_change_percent,per_lesion_change_absolute,sub_lesion_counts,sub_lesion_change_percent,sub_lesion_change_absolute
0,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,-1,"[1, 0]",0.0,-1,"[2, 0]",0.00,-2,"[1, 1]",0.00,0
1,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,24,"[1, 5]",400.0,4,"[2, 4]",100.00,2,"[1, 12]",1100.00,11
2,mk1,0,1,1,1,0,throughout,intermittent,intermittent,intermittent,...,25,"[0, 5]",0.0,5,"[0, 4]",0.00,4,"[1, 12]",1100.00,11
3,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,-1,"[1, 4]",300.0,3,"[2, 5]",150.00,3,"[5, 0]",0.00,-5
4,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,2,"[1, 4]",300.0,3,"[2, 10]",400.00,8,"[5, 9]",80.00,4
5,mk2,0,0,0,0,1,throughout,throughout,throughout,throughout,...,3,"[4, 4]",0.0,0,"[5, 10]",100.00,5,"[0, 9]",0.00,9
6,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,2,"[0, 0]",0.0,0,"[4, 10]",150.00,6,"[0, 11]",0.00,11
7,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,1,"[0, 1]",0.0,1,"[4, 3]",-25.00,-1,"[0, 3]",0.00,3
8,mk3,0,0,2,0,1,throughout,throughout,late,throughout,...,-1,"[0, 1]",0.0,1,"[10, 3]",-70.00,-7,"[11, 3]",-72.73,-8
9,mk4,0,1,0,0,0,throughout,early,throughout,throughout,...,0,"[4, 4]",0.0,0,"[4, 3]",-25.00,-1,"[5, 3]",-40.00,-2


In [47]:
def mse_img(mseid): #gather image for given mseid
    les_src = glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/alignment_lesions.nii.gz" % mseid)
    if len(les_src) == 0:
        les_src = glob.glob("/data/henry7/PBR/subjects/%s/lesions_manual/*/*lesions.nii.gz" % mseid)
        if len(les_src) == 0:
            print "error"
    les_file = les_src[0]
    les_img = nib.load(les_file)
    return les_img

def coords(les_img): #gather coordinates from img function
    les_data = les_img.get_data()
    labels, n_labels = label(les_data==1)
    all_label_coords = []
    for count in range(1,n_labels+1):
        x,y,z = np.nonzero(labels==count)
        cur_coords=[]
        for count in range(len(x)):
            cur_coords.append([x[count],y[count],z[count]])
        all_label_coords.append(cur_coords)
    return all_label_coords

def affines(img1,img2): #gather affines from img function
    les_affine = img1.get_affine()
    conv_affine = np.linalg.inv(img2.get_affine())
    return les_affine,conv_affine

def reformat(coordinates): #turn coordinates from [XXX, YYY, ZZZ] format to 'XXXYYYZZZ' format
    coord_list=[]
    for x in range(len(coordinates)):
        cur_coord_list = []
        for y in range(len(coordinates[x])):
            cur_coord_list.append("%03d%03d%03d" % (coordinates[x][y][0],coordinates[x][y][1],coordinates[x][y][2]))
        coord_list.append(cur_coord_list)
    return coord_list

def transform1to2space(img1,img2): #call coords and affines function for each mseid, and transforms those lesion 
                                   #coordinates to space of more recent mseid
    les = coords(img1)
    untransformed_coords = coords(img2)
    reformat_utf_coords = reformat(untransformed_coords)
    affine, invaffine = affines(img1,img2)
    transformed_coords = []
    print len(les), len(untransformed_coords)
    for x in range(len(les)):
        current_coords=[]
        for y in range(len(les[x])):
            first_transform = np.dot(affine,[les[x][y][0],
                                            les[x][y][1],
                                            les[x][y][2],
                                            1])
            second_transform = np.dot(invaffine,[first_transform[0],
                                                first_transform[1],
                                                first_transform[2],
                                                1])
            current_coords.append([int(second_transform[0]),int(second_transform[1]),int(second_transform[2])])
        transformed_coords.append(current_coords)
    reformat_tf_coords = reformat(transformed_coords)
    return transformed_coords, untransformed_coords, reformat_tf_coords, reformat_utf_coords

mse1 = 'mse1'
mse2 = 'mse1361'
coords1,coords2,rfcoords1,rfcoords2 = transform1to2space(mse_img(mse1),mse_img(mse2))

for x in range(len(rfcoords1)):
    for y in range(len(rfcoords2)):
        overlaps = list(set(rfcoords1[x]) & set(rfcoords2[y]))
        if overlaps != []:
            overlap_vol,les1_vol,les2_vol = len(overlaps),len(rfcoords1[x]),len(rfcoords2[y])
            overlap_x_les1 = round((float(overlap_vol) / float(les1_vol)) * 100, 3)
            overlap_x_les2 = round((float(overlap_vol) / float(les2_vol)) * 100, 3)
            change_absolute = les2_vol - les1_vol
            change_percent = percentchange(les2_vol,les1_vol)
            lesion1type = les_data[les_data['mseID']==mse1][les_data['lesion']==x]['type']
            lesion2type = les_data[les_data['mseID']==mse2][les_data['lesion']==y]['type']
            print (x*(len(rfcoords2)) + y), x, lesion1type, y, lesion2type, overlap_vol, les1_vol,les2_vol,overlap_x_les1,overlap_x_les2, change_absolute, change_percent
            print overlaps
            

15 4
46 11 223    periventricular
Name: type, dtype: object 2 98    subcortical
Name: type, dtype: object 30 72 372 41.667 8.065 300 416.67
['161076059', '161076058', '161077058', '161077059', '160074059', '162075058', '159075059', '159075058', '161075058', '161075059', '160076058', '160076059', '161074059', '159076059', '161076060', '162076057', '159074060', '160075059', '160075058', '159075060', '159075061', '162076058', '160074061', '160074060', '160076060', '161075060', '160075060', '160075061', '161074060', '159076061']


/data/henry1/keshavan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:74: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/data/henry1/keshavan/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:75: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
